In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
TARGET_FOLDER = Path('../../data/features/')

**Create a Vector for each node, the vector represents tfs that it's connected to**
keep in mind
1. tf2tf is directional
2. tf2gene is directional

In [5]:
DATA_FOLDER = Path('../../data/')

# 1. Read edgelist
df = pd.read_csv(DATA_FOLDER / 'joint_renamed_v2.csv', dtype={
    'cell_type': object,
    'source': object,
    'target': object,
    'type': object,
    'weight': 'float64'
})
df

,cell_type,source,target,type,weight
0,GM12878,ATF3_gm,SDF4,TSS,179.618811
1,GM12878,ATF3_gm,B3GALT6,TSS,178.254494
2,GM12878,ATF3_gm,SSU72,TSS,413.000000
3,GM12878,ATF3_gm,AL645728.1,TSS,413.000000
4,GM12878,ATF3_gm,CDK11A,TSS,160.000000
...,...,...,...,...,...
777916,NaN,ZYX,NEDD8,PPI,NaN
777917,NaN,ZYX,UBC,PPI,NaN
777918,NaN,ZYX,VASP,PPI,NaN
777919,NaN,ZZEF1,UBC,PPI,NaN


In [22]:
df[df['source'] == 'ATF3']

,cell_type,source,target,type,weight
737438,NaN,ATF3,ATF2,PPI,NaN
737439,NaN,ATF3,ATF3,PPI,NaN
737440,NaN,ATF3,ATF3,PPI,NaN
737441,NaN,ATF3,ATF4,PPI,NaN
737442,NaN,ATF3,DDIT3,PPI,NaN
737443,NaN,ATF3,HDAC1,PPI,NaN
737444,NaN,ATF3,JUN,PPI,NaN
737445,NaN,ATF3,MDM2,PPI,NaN
737446,NaN,ATF3,SMAD3,PPI,NaN
737447,NaN,ATF3,SMARCA4,PPI,NaN


In [6]:
# read common tfs
common_tf = list(set(pd.read_csv(DATA_FOLDER / 'common_tf.csv')['tf']))
common_tf_k = [tf + '_k' for tf in common_tf]
common_tf_gm = [tf + '_gm' for tf in common_tf]
common_tf = common_tf_k + common_tf_gm
common_tf.sort()
len(common_tf)

138

In [7]:
a = df['source'] == node
b = df['target'].isin(common_tf)
a & b

NameError: name 'node' is not defined

In [30]:
a = np.array([1, 2, 3, 4])
a[[0, 2, 3]] += 10
a

array([11,  2, 13, 14])

In [9]:
nodes = list(set(df['source']).union(set(df['target'])))
len(nodes)

18546

In [10]:
d = dict()
for i, tf in enumerate(common_tf):
    d[tf] = i


def node2neighbors(node, df=df):
    feature = np.array([0] * len(common_tf))
    a = df['target'] == node
    b = df['source'].isin(common_tf)
    regulators = list(df[a & b]['source'])
    linked_tf_pos = list(map(lambda tf: d[tf], regulators))
    feature[linked_tf_pos] = 1
    return feature


features = dict(zip(nodes, map(node2neighbors, nodes)))    

In [21]:
feature_df = pd.DataFrame.from_dict(features, orient='index')
feature_df.loc['ATF3_gm']

0      0
1      0
2      1
3      0
4      1
      ..
133    0
134    1
135    0
136    0
137    0
Name: ATF3_gm, Length: 138, dtype: int64

In [24]:
feature_df.loc[~feature_df.index.isin(common_tf)]

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
TRIM59,0,0,1,1,0,1,1,1,0,1,...,0,0,0,1,1,1,1,1,0,0
HAUS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
DKFZP761J1410,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
ABI1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,1,1,0,0
CCDC15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OVCH1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PES1,0,1,1,0,1,1,0,0,0,0,...,0,0,0,1,1,1,1,1,0,0
SAMD1,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,1,1,1,1,0,0
ALG14,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,1,1,1,0,0


In [14]:
# test = pd.DataFrame.from_dict({
#     'A': [0, 0, 0, 1],
#     'B': [1, 1, 1, 1]
# }, orient='index')
# test['count_1'] = test.apply(lambda x: sum(x), axis=1)
# test

feature_df['count_1'] = feature_df.sum(axis=1)
feature_df[feature_df['count_1'] == 0]

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,count_1
HSFY1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TUBB7P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CSNK2A1P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCF1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRYGC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AZGP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H2BFS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
KRTAP3-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
test = pd.read_csv(DATA_FOLDER / 'joint_renamed_v2.csv')
test

,cell_type,source,target,type,weight
0,GM12878,ATF3_gm,SDF4,TSS,179.618811
1,GM12878,ATF3_gm,B3GALT6,TSS,178.254494
2,GM12878,ATF3_gm,SSU72,TSS,413.000000
3,GM12878,ATF3_gm,AL645728.1,TSS,413.000000
4,GM12878,ATF3_gm,CDK11A,TSS,160.000000
...,...,...,...,...,...
777916,NaN,ZYX,NEDD8,PPI,NaN
777917,NaN,ZYX,UBC,PPI,NaN
777918,NaN,ZYX,VASP,PPI,NaN
777919,NaN,ZZEF1,UBC,PPI,NaN


In [35]:
G = test[test['cell_type'] == 'GM12878']
G

,cell_type,source,target,type,weight
0,GM12878,ATF3_gm,SDF4,TSS,179.618811
1,GM12878,ATF3_gm,B3GALT6,TSS,178.254494
2,GM12878,ATF3_gm,SSU72,TSS,413.000000
3,GM12878,ATF3_gm,AL645728.1,TSS,413.000000
4,GM12878,ATF3_gm,CDK11A,TSS,160.000000
...,...,...,...,...,...
350469,GM12878,JUND_gm,G6PD,TSS,401.905464
350470,GM12878,JUND_gm,VAMP7,TSS,823.020345
350471,GM12878,JUND_gm,CRLF2,TSS,1000.000000
350472,GM12878,JUND_gm,ASMTL,TSS,1363.003897


In [39]:
G_target = set(G['target'])
K_target = set(K['target'])

In [42]:
ppi = pd.read_csv(DATA_FOLDER / 'ppi_undirected.csv')

In [43]:
gene = set(ppi['source']).union(set(ppi['target']))

In [46]:
len(G_target.intersection(gene))

7791

In [47]:
len(K_target.intersection(gene))

7889